In [58]:
import json
import os
import pandas as pd

# Define path
file_path = r"C:\Users\kyram\OneDrive\School\Prepossesed Data.json"

# Confirm the file exists
if not os.path.exists(file_path):
    print(f"The file does not exist at the specified path: {file_path}")
else:
    # Load the JSON 
    with open(file_path, 'r') as file:
        preprocessed_data = json.load(file)

    # Print the keys
    print(preprocessed_data.keys())

    # Access the key
    if 'public' in preprocessed_data:
        public_data = preprocessed_data['public']
        # Convert the dictionary back to a DataFrame
        df = pd.DataFrame.from_dict(public_data)
        print(df)
    else:
        print("'public' key not found in the JSON file")

dict_keys(['public'])
       Age   AsAm  AmerInd  AvgDrinksPerWeek  Black  DrinkDaysPerWeek  \
0     83.0  False      0.0               0.0    0.0               0.0   
1     58.0  False      0.0              35.0    0.0               7.0   
2     40.0  False      0.0               0.0    0.0               0.0   
3     66.0  False      0.0               0.0    0.0               0.0   
4     39.0  False      0.0               0.0    1.0               0.0   
...    ...    ...      ...               ...    ...               ...   
6247  48.0  False      0.0               4.0    0.0               2.0   
6248  50.0  False      0.0               0.0    0.0               0.0   
6249  55.0  False      0.0               3.0    1.0               1.0   
6250  62.0  False      0.0              84.0    0.0               7.0   
6251  57.0  False      0.0               0.0    0.0               0.0   

      DrinksOneOccasion  DrinksPerDay  Education  EverHadCancer  ...  \
0                   0.0      

In [59]:
# Import the necessary libraries
import numpy as np
import random
import sys
import tensorflow 

from collections import Counter

from imblearn.over_sampling import RandomOverSampler

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [60]:
# Define X and y
X = df.copy().drop(['EverHadCancer'], axis=1)

y = df['EverHadCancer']

In [61]:
# Encode y
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

# Impute X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

In [62]:
# Scale X
numerical_transformer = StandardScaler()

X_imputed = numerical_transformer.fit_transform(X_imputed)

In [63]:
# Split into train and test splits
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_numeric, test_size=0.2, random_state=42, stratify=y_numeric)

# Resample
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [64]:
# Check shapes and distributions
print("Shape:", df.shape)
print("Dataset distribution:", Counter(df))

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("y_test Dataset distribution:", Counter(y_test))

print("X__train_resampled Shape", X_train_resampled.shape)
print("y_train_resampled Shape", y_train_resampled.shape)
print("y_train_resampled Dataset distribution:", Counter(y_train_resampled))

Shape: (6252, 59)
Dataset distribution: Counter({'Age': 1, 'AsAm': 1, 'AmerInd': 1, 'AvgDrinksPerWeek': 1, 'Black': 1, 'DrinkDaysPerWeek': 1, 'DrinksOneOccasion': 1, 'DrinksPerDay': 1, 'Education': 1, 'EverHadCancer': 1, 'FamilyEverHadCancer2': 1, 'FreqGoProvider': 1, 'HealthInsurance2': 1, 'HeardHPVVaccine2': 1, 'HPVCauseCancer_Cervical': 1, 'Hisp': 1, 'IncomeRanges': 1, 'MedConditions_Diabetes': 1, 'MedConditions_HighBP': 1, 'MedConditions_HeartCondition': 1, 'MedConditions_LungDisease': 1, 'OthPacIsl': 1, 'Smoke100': 1, 'Sunburned_Active': 1, 'Sunburned_Other': 1, 'Sunburned_Rec': 1, 'Sunburned_Work': 1, 'TimesSunburned': 1, 'UsedECigEver': 1, 'WhenPapTest': 1, 'White': 1, 'BirthGender_Female': 1, 'BirthGender_Male': 1, 'DelayNeededCare_I did not need any medical care in the past 12 months': 1, 'DelayNeededCare_No, I received the medical care I felt I needed': 1, 'DelayNeededCare_Yes': 1, 'DocTalkLDCT_Dont know': 1, 'DocTalkLDCT_I have never heard of this test': 1, 'DocTalkLDCT_No':

In [65]:
# Set seed
random.seed(42)

# Define model
model_1 = Sequential([
    Dense(58, activation='relu', input_shape=(58,)),  
    Dense(30, activation='relu'), 
    Dense(len(np.unique(y)), activation='softmax') 
])


# Compile model 
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_1.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_1, accuracy_1 = model_1.evaluate(X_test, y_test)
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')

# Predict y
y_pred_1 = model_1.predict(X_test)
y_pred_classes_1 = np.argmax(y_pred_1, axis=1)

# F1 score and confusion matrix
f1_1 = f1_score(y_test, y_pred_classes_1, average='weighted')
conf_matrix_1 = confusion_matrix(y_test, y_pred_classes_1)

print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)


Epoch 1/100
215/215 [==============================] - 1s 3ms/step - loss: 0.5699 - accuracy: 0.6839 - val_loss: 0.7648 - val_accuracy: 0.5301
Epoch 2/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4665 - accuracy: 0.7641 - val_loss: 0.6334 - val_accuracy: 0.6743
Epoch 3/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4168 - accuracy: 0.8042 - val_loss: 0.7207 - val_accuracy: 0.6042
Epoch 4/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3784 - accuracy: 0.8271 - val_loss: 0.4934 - val_accuracy: 0.7823
Epoch 5/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3453 - accuracy: 0.8483 - val_loss: 0.4525 - val_accuracy: 0.7998
Epoch 6/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3122 - accuracy: 0.8683 - val_loss: 0.4292 - val_accuracy: 0.8179
Epoch 7/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2855 - accuracy: 0.8828 - val_loss: 0.4084 - val_accuracy: 0.8260

In [66]:
# Set seed
random.seed(42)

# Define model
model_2 = Sequential([
    Dense(58, activation='relu', input_shape=(58,)),  
    Dense(30, activation='relu'), 
    Dense(15, activation='relu'), 
    Dense(len(np.unique(y)), activation='softmax') 
])


# Compile model 
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_2.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_2, accuracy_2 = model_2.evaluate(X_test, y_test)
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')

# Predict y
y_pred_2 = model_2.predict(X_test)
y_pred_classes_2 = np.argmax(y_pred_2, axis=1)

# F1 score and confusion matrix
f1_2 = f1_score(y_test, y_pred_classes_2, average='weighted')
conf_matrix_2 = confusion_matrix(y_test, y_pred_classes_2)

print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)


Epoch 1/100
215/215 [==============================] - 1s 3ms/step - loss: 0.5578 - accuracy: 0.6795 - val_loss: 0.7785 - val_accuracy: 0.5820
Epoch 2/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4537 - accuracy: 0.7681 - val_loss: 0.5123 - val_accuracy: 0.8132
Epoch 3/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4006 - accuracy: 0.8045 - val_loss: 0.5393 - val_accuracy: 0.7758
Epoch 4/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3496 - accuracy: 0.8417 - val_loss: 0.4629 - val_accuracy: 0.8132
Epoch 5/100
215/215 [==============================] - 0s 1ms/step - loss: 0.3036 - accuracy: 0.8733 - val_loss: 0.4022 - val_accuracy: 0.8511
Epoch 6/100
215/215 [==============================] - 0s 1ms/step - loss: 0.2642 - accuracy: 0.8906 - val_loss: 0.2850 - val_accuracy: 0.9054
Epoch 7/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2254 - accuracy: 0.9128 - val_loss: 0.2502 - val_accuracy: 0.9305

In [67]:
# Set seed
random.seed(42)

# Define model
model_3 = Sequential([
    Dense(58, activation='relu', input_shape=(58,)),  
    Dense(40, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y)), activation='softmax') 
])


# Compile model 
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_3.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_3, accuracy_3 = model_3.evaluate(X_test, y_test)
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')

# Predict y
y_pred_3 = model_3.predict(X_test)
y_pred_classes_3 = np.argmax(y_pred_3, axis=1)

# F1 score and confusion matrix
f1_3 = f1_score(y_test, y_pred_classes_3, average='weighted')
conf_matrix_3 = confusion_matrix(y_test, y_pred_classes_3)

print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)


Epoch 1/100
215/215 [==============================] - 1s 2ms/step - loss: 0.5452 - accuracy: 0.7062 - val_loss: 0.7493 - val_accuracy: 0.6404
Epoch 2/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4523 - accuracy: 0.7705 - val_loss: 0.5935 - val_accuracy: 0.7682
Epoch 3/100
215/215 [==============================] - 0s 1ms/step - loss: 0.3939 - accuracy: 0.8144 - val_loss: 0.4542 - val_accuracy: 0.8529
Epoch 4/100
215/215 [==============================] - 0s 1ms/step - loss: 0.3433 - accuracy: 0.8487 - val_loss: 0.4138 - val_accuracy: 0.8622
Epoch 5/100
215/215 [==============================] - 0s 1ms/step - loss: 0.2940 - accuracy: 0.8765 - val_loss: 0.5117 - val_accuracy: 0.7752
Epoch 6/100
215/215 [==============================] - 0s 1ms/step - loss: 0.2556 - accuracy: 0.8971 - val_loss: 0.3843 - val_accuracy: 0.8640
Epoch 7/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2075 - accuracy: 0.9201 - val_loss: 0.2510 - val_accuracy: 0.9165

In [68]:
# Set seed
random.seed(42)

# Define model
model_4 = Sequential([
    Dense(58, activation='relu', input_shape=(58,)),  
    Dense(40, activation='relu'), 
    Dense(30, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_4.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_4, accuracy_4 = model_4.evaluate(X_test, y_test)
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')

# Predict y
y_pred_4 = model_4.predict(X_test)
y_pred_classes_4 = np.argmax(y_pred_4, axis=1)

# F1 score and confusion matrix
f1_4 = f1_score(y_test, y_pred_classes_4, average='weighted')
conf_matrix_4 = confusion_matrix(y_test, y_pred_classes_4)

print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)


Epoch 1/100
215/215 [==============================] - 1s 3ms/step - loss: 0.5614 - accuracy: 0.6759 - val_loss: 0.8119 - val_accuracy: 0.6159
Epoch 2/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4561 - accuracy: 0.7795 - val_loss: 0.5608 - val_accuracy: 0.8050
Epoch 3/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3885 - accuracy: 0.8236 - val_loss: 0.3802 - val_accuracy: 0.9037
Epoch 4/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3278 - accuracy: 0.8565 - val_loss: 0.4050 - val_accuracy: 0.8838
Epoch 5/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2796 - accuracy: 0.8847 - val_loss: 0.4996 - val_accuracy: 0.7910
Epoch 6/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2298 - accuracy: 0.9101 - val_loss: 0.4713 - val_accuracy: 0.8208
Epoch 7/100
215/215 [==============================] - 0s 2ms/step - loss: 0.1952 - accuracy: 0.9225 - val_loss: 0.2648 - val_accuracy: 0.9200

In [69]:
# Compare the differnet versions of the model
print('Model_1: 1 layered model')
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')
print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)

print('Model_2: 2 layered model')
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')
print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)

print('Model_3: 3 layered model')
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')
print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)

print('Model_4: 4 layered model')
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')
print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)

Model_1: 1 layered model
Loss: 2.2430384159088135, Accuracy: 0.7873700857162476
F1 Score: 0.7888143344230492
Confusion Matrix:
[[935 136]
 [130  50]]
Model_2: 2 layered model
Loss: 2.530100107192993, Accuracy: 0.808952808380127
F1 Score: 0.8023967727242451
Confusion Matrix:
[[965 106]
 [133  47]]
Model_3: 3 layered model
Loss: 2.305609941482544, Accuracy: 0.7905675172805786
F1 Score: 0.7885772734877636
Confusion Matrix:
[[944 127]
 [135  45]]
Model_4: 4 layered model
Loss: 1.9582672119140625, Accuracy: 0.8137490153312683
F1 Score: 0.8035289286524417
Confusion Matrix:
[[975  96]
 [137  43]]


In [70]:
# Define the feature importance
weights = model_4.get_weights()

feature_importance = np.sum(np.abs(weights[0]), axis=1)

In [72]:
# Define the features
features = df.drop(['EverHadCancer'], axis=1).columns

# Convert feature importance to a DataFrame
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': feature_importance
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print(feature_importance_df)


                                              feature  importance
0                                                 Age   12.368540
28                                        WhenPapTest   12.211391
22                                   Sunburned_Active   11.823321
15                                       IncomeRanges   11.203971
12                                   HeardHPVVaccine2   11.077266
11                                   HealthInsurance2   10.944157
39  DocTellColorectalTests_I have never discussed ...   10.715882
16                             MedConditions_Diabetes   10.595102
13                            HPVCauseCancer_Cervical   10.518160
8                                           Education   10.475604
29                                              White   10.292983
44                                   QualityCare_Good   10.287570
24                                      Sunburned_Rec   10.284148
9                                FamilyEverHadCancer2   10.247767
10        

In [73]:
# Convert DataFrame to a dictionary
FNN_importance = {
    'public': feature_importance_df.to_dict(),  
}
# Create path
save_path = r"C:\Users\kyram\OneDrive\School\FNN3_importance.json"

# Confirm the dictionary exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the data to a JSON file
with open(save_path, 'w') as file:
    json.dump(FNN_importance, file)

print(f"Data successfully saved to {save_path}")

Data successfully saved to C:\Users\kyram\OneDrive\School\FNN3_importance.json
